In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/datasets/satellite"

dubai_dataset


In [ ]:
!pip install patchify

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import pyplot as plt
import random

In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
dataset_root_folder = '/content/drive/MyDrive/Colab Notebooks/datasets/satellite/'

In [ ]:
dataset_name = "dubai_dataset"

In [ ]:
image_patch_size = 256

In [ ]:
image_dataset = []
mask_dataset = []

In [ ]:
for image_type in ['images', 'masks']:
    image_extension = 'jpg' if image_type == 'images' else 'png'

    for tile_id in range(1, 8):
        for image_id in range(1, 20):
            file_path = f"{dataset_root_folder}/{dataset_name}/Tile {tile_id}/{image_type}"
            image = cv2.imread(file_path, 1)

            if image is None:
                continue  # skip missing files

            if image_type == 'masks':
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # crop image to patch multiple of 256
            size_x = (image.shape[1] // image_patch_size) * image_patch_size
            size_y = (image.shape[0] // image_patch_size) * image_patch_size
            image = Image.fromarray(image)
            image = image.crop((0, 0, size_x, size_y))
            image = np.array(image)

            # patchify differently for images vs masks
            if image_type == 'images':
                patches = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
            else:
                # for masks (same shape)
                patches = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)

            for i in range(patches.shape[0]):
                for j in range(patches.shape[1]):
                    patch = patches[i, j, 0]  # select patch properly

                    if image_type == 'images':
                        # Normalize each patch
                        patch = minmaxscaler.fit_transform(patch.reshape(-1, patch.shape[-1])).reshape(patch.shape)
                        image_dataset.append(patch)
                    else:
                        mask_dataset.append(patch)

In [ ]:
image_dataset = np.array(image_dataset)
mask_dataset = np.array(mask_dataset)

In [ ]:
print(len(image_dataset))
print(len(mask_dataset))

0
0
